In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# !pip install diagonal-b6
# !pip install pandas
# !pip install geopandas
# !pip install shapely

In [ ]:
import diagonal_b6 as b6
import pandas as pd

import geopandas as gpd
from shapely.geometry import shape
from shapely.geometry.polygon import Polygon

In [ ]:
# connect to the world
grpc_address = "localhost:8002"
w = b6.connect_insecure(grpc_address)

In [ ]:
# using a small area in Camden town, for demonstration
CAMDEN_WKT = "POLYGON ((-0.15258173047936907 51.54625419866031, -0.15258173047936907 51.53907407833145, -0.12912255595443867 51.53907407833145, -0.12912255595443867 51.54625419866031, -0.15258173047936907 51.54625419866031))"
camden_polygon = b6.wkt(CAMDEN_WKT)

Find benches

Then find how many toilets are nearby each bench

In [ ]:
%%time

benches_in_camden = b6.find_points( b6.intersecting(camden_polygon).and_(b6.tagged("#amenity","bench")) )

for osmid, feature in w(benches_in_camden):
    print( f"{osmid}\n{feature.all_tags()}" )

In [ ]:
%%time

reachable_toilets = benches_in_camden.map( lambda b: b.reachable("walk",
                                                                500,
                                                                b6.tagged("#amenity","toilets")) \
                                                    .count()
                                        )

df = pd.DataFrame([(b.value, c) for b,c in w(reachable_toilets)], 
                  columns=["bench","toilet_count"]) \
       .set_index("bench")
df.plot.hist();

What if?

What happens if every parking lot had a toilet?

In [ ]:
%%time

parkinglots = b6.find_areas(b6.tagged("#amenity","parking"))
parkinglots_to_toilets = b6.add_tags( parkinglots.map( lambda parkinglot: b6.tag("#amenity","toilets") )
                                    )

improved_reachable_toilets = b6.with_change(parkinglots_to_toilets, lambda: reachable_toilets)

df_improved = pd.DataFrame([(b.value, c) for b,c in w(improved_reachable_toilets)],
                             columns =["bench","toilet_count"]) \
                .set_index("bench")
df_improved.plot.hist()

In [ ]:
change = df.merge(df_improved, right_index=True, left_index=True, suffixes=["","_improved"])
change.plot.hist(alpha=0.5, bins=max(change["toilet_count_improved"])+1)

Find benches with unobstructed views

In [ ]:
# busywork for later
df["url"] = df.index.map(lambda x: f'<a href="https://www.openstreetmap.org/node/{x}" rel="noopener noreferrer" target="_blank">{x}</a>')
df = df.reindex(columns=["url","toilet_count"])

In [ ]:
%%time

benches_sightline_areas = benches_in_camden.map( lambda b: b.sightline(100).area() )

df_sightline_areas = pd.DataFrame([(b.value, c) for b,c in w(benches_sightline_areas)],
                            columns=["benches","sightline_area"]) \
                       .set_index("benches")

In [ ]:
benches = df.merge(df_sightline_areas, right_index=True, left_index=True)

In [ ]:
benches.loc[benches["toilet_count"]>3] \
       .sort_values(by = ["sightline_area","toilet_count"], ascending=False) \
       .head(20) \
       .style

Sneak peak at the polygons

In [ ]:
%%time

benches_sightline = benches_in_camden.map( lambda b: b6.to_geojson(b.sightline(100)) )
df_sightline = pd.DataFrame([(b.value, shape(c["geometry"])) for b,c in w(benches_sightline)],
                            columns=["benches","geometry"]) \
                 .set_index("benches")

In [ ]:
benches = gpd.GeoDataFrame(benches.merge(df_sightline,
                                         right_index=True,
                                         left_index=True),
                           geometry="geometry")


In [ ]:
benches.style

To see the sightline polygons in the map https://http://localhost:8001/

Use `` ` `` to pop out the terminal then, using the appropriate OSM ID (the example uses 7744625385) run

```
find-feature /point/openstreetmap.org/node/7744625385 | sightline 100
```